In [ ]:
import re
import pandas as pd

from horsePlayground import parseFullDay
from horseInfoFns import parseHorseInfo
from genInfoFns import parseGenInfo
from timesInfoFns import parseTimeInfo
from betInfoFns import parseBetInfo
from runlineInfoFns import parseRunlineInfo

In [ ]:
with open('./../charts/chartsTxt/eqbPDFChartPlus - 2020-08-11T010651.112.txt') as file:
    test1 = file.readlines()
with open('./../charts/chartsTxt/eqbPDFChartPlus - 2020-08-11T010651.148.txt') as file:
    test2 = file.readlines()

In [ ]:
jack = parseFullDay(test2)

In [11]:
def parseTrainerLine(line):
    fullSearch = re.search(r'^ Trainers: (\d?\d - [^;]+ )+', line)
    numGroups = len(fullSearch.groups())

    pgm = []
    name = []

    for i in range(1, 1+numGroups):
        shortSearch = re.search(r'(\d?\d) - (.+) $', fullSearch(i))

        pgm.append(shortSearch.group(1))
        name.append(shortSearch.group(2))

    return [pgm, name]

In [12]:
line = " Trainers: 6 - Sherlock, Gary; 5 - O'Dell, Christopher; 3 - Gomez, Ruben; 1 - Stortz, Marcia; 4 - Gonzalez, Felix; 2 - Mendoza, Jesus"

In [13]:
parseTrainerLine(line)

AttributeError: 'tuple' object has no attribute 'groups'